# Homework 2 - Data Wrangling

In [1]:
import pandas as pd

/Users/ivy/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Data

In [4]:
cdi = pd.read_csv("~/Li_Xinran/hw2/U.S._Chronic_Disease_Indicators__CDI_.csv")
cdi.head()

/Users/ivy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


YearStart  YearEnd LocationAbbr LocationDesc         DataSource   Topic  \
0       2010     2014           AK       Alaska  Death Certificate  Cancer   
1       2010     2014           AK       Alaska  Death Certificate  Cancer   
2       2010     2014           AZ      Arizona  Death Certificate  Cancer   
3       2008     2012           NY     New York  Death Certificate  Cancer   
4       2009     2013           NY     New York  Death Certificate  Cancer   

                                           Question  Response DataValueUnit  \
0  Cancer of the oral cavity and pharynx, mortality       NaN           NaN   
1  Cancer of the oral cavity and pharynx, mortality       NaN           NaN   
2  Cancer of the oral cavity and pharynx, mortality       NaN           NaN   
3  Cancer of the oral cavity and pharynx, mortality       NaN           NaN   
4  Cancer of the oral cavity and pharynx, mortality       NaN           NaN   

           DataValueType  ... LocationID  TopicID QuestionID DataValueTypeID  \
0  Average Annual Number  ...          2      CAN    CAN10_2      AVGANNNMBR   
1  Average Annual Number  ...          2      CAN    CAN10_2      AVGANNNMBR   
2  Average Annual Number  ...          4      CAN    CAN10_2      AVGANNNMBR   
3  Average Annual Number  ...         36      CAN    CAN10_2      AVGANNNMBR   
4  Average Annual Number  ...         36      CAN    CAN10_2      AVGANNNMBR   

   StratificationCategoryID1  StratificationID1 StratificationCategoryID2  \
0                     GENDER               GENF                       NaN   
1                    OVERALL                OVR                       NaN   
2                     GENDER               GENM                       NaN   
3                     GENDER               GENM                       NaN   
4                     GENDER               GENM                       NaN   

  StratificationID2  StratificationCategoryID3  StratificationID3  
0               NaN                        NaN                NaN  
1               NaN                        NaN                NaN  
2               NaN                        NaN                NaN  
3               NaN                        NaN                NaN  
4               NaN                        NaN                NaN  

[5 rows x 34 columns]

## Selection of Data and Reshaping the Data

In [5]:
#select poverty & binge >= 18 yrs with crude prevalence and stratification as overall and F/M
cdi1 = cdi[(cdi['Question'].isin(['Poverty','Binge drinking prevalence among adults aged >= 18 years'])) &
            (cdi['DataValueType'] == 'Crude Prevalence') &
          (cdi["Stratification1"].isin(["Overall", "Female", "Male"]))]

#only select overall for poverty
cdi2 = cdi1[((cdi1["Question"] == "Poverty") & (cdi1["Stratification1"] == "Overall"))|
               (cdi1["Question"] == "Binge drinking prevalence among adults aged >= 18 years")]

#select to keep useful columns only
CDI = cdi2.loc[:,["YearStart", "LocationAbbr", "LocationDesc", "Question", "DataValueType",
            "DataValue", "Stratification1"]]
CDI.head(5)
#CDI.shape

YearStart LocationAbbr  LocationDesc Question     DataValueType  \
340       2015           AL       Alabama  Poverty  Crude Prevalence   
369       2014           TX         Texas  Poverty  Crude Prevalence   
379       2011           NM    New Mexico  Poverty  Crude Prevalence   
473       2011           WY       Wyoming  Poverty  Crude Prevalence   
489       2011           ND  North Dakota  Poverty  Crude Prevalence   

    DataValue Stratification1  
340      18.5         Overall  
369      17.2         Overall  
379      21.5         Overall  
473      11.3         Overall  
489      12.2         Overall

In [6]:
#convert to wide format
cdi_wide = CDI.pivot(index = ["LocationDesc","LocationAbbr","YearStart"], 
                           columns = ["Question","Stratification1"], 
                           values = "DataValue").reset_index()

#joining multiindex colnames
cdi_wide.columns = [" ".join(col) for col in cdi_wide.columns.values]
cdi_wide.head(5)

LocationDesc  LocationAbbr   YearStart  Poverty Overall  \
0       Alabama            AL        2010              19   
1       Alabama            AL        2011              19   
2       Alabama            AL        2012              19   
3       Alabama            AL        2013            18.7   
4       Alabama            AL        2014            19.3   

  Binge drinking prevalence among adults aged >= 18 years Female  \
0                                                NaN               
1                                                7.9               
2                                                  7               
3                                                6.5               
4                                                6.8               

  Binge drinking prevalence among adults aged >= 18 years Male  \
0                                                NaN             
1                                               20.2             
2                                               18.2             
3                                               16.3             
4                                               17.9             

  Binge drinking prevalence among adults aged >= 18 years Overall  
0                                                NaN               
1                                               13.7               
2                                               12.3               
3                                               11.2               
4                                                 12

In [7]:
#rename column variables
cdi_clean = cdi_wide.rename(columns = {"LocationDesc ":"state", "LocationAbbr ":"stateabb",
                                       "YearStart ":"year", "Poverty Overall":"poverty", 
                                       "Binge drinking prevalence among adults aged >= 18 years Female":"binge_female",
                                      "Binge drinking prevalence among adults aged >= 18 years Male":"binge_male",
                                      "Binge drinking prevalence among adults aged >= 18 years Overall":"binge_all"})
#change data type to numerical
cdi_clean["year"] = cdi_clean["year"].astype("int")
cdi_clean["poverty"] = cdi_clean["poverty"].astype("float")
cdi_clean["binge_female"] = cdi_clean["binge_female"].astype("float")
cdi_clean["binge_male"] = cdi_clean["binge_male"].astype("float")
cdi_clean["binge_all"] = cdi_clean["binge_all"].astype("float")

cdi_clean.head()

state stateabb  year  poverty  binge_female  binge_male  binge_all
0  Alabama       AL  2010     19.0           NaN         NaN        NaN
1  Alabama       AL  2011     19.0           7.9        20.2       13.7
2  Alabama       AL  2012     19.0           7.0        18.2       12.3
3  Alabama       AL  2013     18.7           6.5        16.3       11.2
4  Alabama       AL  2014     19.3           6.8        17.9       12.0

In [8]:
cdi_clean.shape

(548, 7)

In [9]:
cdi_clean.describe()

year     poverty  binge_female  binge_male   binge_all
count   548.000000  477.000000    486.000000  486.000000  486.000000
mean   2014.516423   14.935220     11.761111   22.474280   16.968313
std       2.867218    5.227848      2.838002    3.729218    3.116518
min    2010.000000    7.300000      5.000000   13.100000    9.600000
25%    2012.000000   11.800000      9.900000   20.200000   15.100000
50%    2015.000000   14.000000     11.800000   22.100000   16.800000
75%    2017.000000   16.900000     13.200000   24.300000   18.700000
max    2019.000000   46.200000     22.300000   33.800000   27.200000

In [10]:
cdi_clean.to_csv("binge_clean.csv")

## Data Transformation and Summary Results

Produce a table that shows the overall, female, and male binge drinking prevalences across U.S. States in the most recent year of data for the Top 10 binge drinking states (i.e. the ones with the highest prevalence in the overall population). Use the relevant pandas commands to select the right variables, sort the data, and filter the data frame.

In [56]:
table1 = cdi_clean[cdi_clean["year"] == 2019].nlargest(10,"binge_all", keep = "all").drop(["poverty","year"],axis=1).set_index("state")

table1

stateabb  binge_female  binge_male  binge_all
state                                                             
District of Columbia       DC          22.3        27.8       24.9
North Dakota               ND          14.9        29.2       22.2
Wisconsin                  WI          16.5        27.7       21.9
Iowa                       IA          16.5        27.0       21.6
South Dakota               SD          16.1        26.4       21.2
Nebraska                   NE          15.7        26.1       20.9
Minnesota                  MN          16.3        25.3       20.7
Montana                    MT          14.7        26.7       20.7
Illinois                   IL          14.4        25.8       19.9
Massachusetts              MA          15.4        23.2       19.1

Calculate the average annual growth rates (in percent) of overall binge drinking across states for the years the data is available. One way to get these growth rates, is to group the data by state (groupby) and use the first() and last() commands to get the first and last non-NA percentage followed by dividing the calculated percentage increase by the number of years data is available for. Alternatively, you could use the pct_change function to help you out. Provide a table of the 5 states with the largest increases and the 5 states with the largest decreases in binge drinking prevalence over the time period.

In [137]:
table2 = cdi_clean[["state","year", "binge_all"]].dropna()
#.pivot(index = "state", columns = "year", values = "binge_all")

#columns=["2011","2012","2013","2014","2015","2016","2017","2018","2019"]
#table2.columns=columns
#table2["avg_annual_growth_rates"] = (table2["2019"] - table2["2011"])/9

table2.head()
table2.groupby("state").last()

year  binge_all
state                                
Alabama               2019       12.4
Alaska                2019       15.9
Arizona               2019       15.1
Arkansas              2019       13.7
California            2019       16.8
Colorado              2019       18.2
Connecticut           2019       15.6
Delaware              2019       17.2
District of Columbia  2019       24.9
Florida               2019       15.8
Georgia               2019       15.7
Guam                  2019       18.0
Hawaii                2019       17.3
Idaho                 2019       16.6
Illinois              2019       19.9
Indiana               2019       15.5
Iowa                  2019       21.6
Kansas                2019       17.4
Kentucky              2019       15.4
Louisiana             2019       18.9
Maine                 2019       17.0
Maryland              2019       13.7
Massachusetts         2019       19.1
Michigan              2019       17.9
Minnesota             2019       20.7
Mississippi           2019       13.4
Missouri              2019       16.8
Montana               2019       20.7
Nebraska              2019       20.9
Nevada                2019       16.8
New Hampshire         2019       16.7
New Jersey            2018       13.5
New Mexico            2019       14.7
New York              2019       16.7
North Carolina        2019       13.5
North Dakota          2019       22.2
Ohio                  2019       17.5
Oklahoma              2019       12.7
Oregon                2019       17.5
Pennsylvania          2019       16.9
Puerto Rico           2019       13.1
Rhode Island          2019       18.2
South Carolina        2019       15.9
South Dakota          2019       21.2
Tennessee             2019       14.4
Texas                 2019       17.9
United States         2019       16.8
Utah                  2019       11.2
Vermont               2019       18.3
Virgin Islands        2016       15.2
Virginia              2019       15.3
Washington            2019       14.1
West Virginia         2019       12.4
Wisconsin             2019       21.9
Wyoming               2019       17.7

In [ ]:
largest_i = table2.nlargest(5, columns = "avg_annual_growth_rates")
largest_i

In [128]:
largest_d = table2.nsmallest(5, columns = "avg_annual_growth_rates")
largest_d

2011  2012  2013  2014  2015  2016  2017  2018  2019  \
state                                                              
Alaska      20.8  16.8  19.5  20.2  20.3  18.2  19.7  15.9  15.9   
Maryland    18.0  16.4  14.2  15.4  14.2  15.3  15.3  13.6  13.7   
Hawaii      21.5  18.2  18.3  19.7  18.9  18.6  19.5  19.2  17.3   
Oklahoma    16.5  14.4  12.7  12.8  13.1  11.8  13.4  13.0  12.7   
Washington  17.8  16.9  16.8  17.1  16.1  16.4  15.6  14.7  14.1   

            avg_annual_growth_rates  
state                                
Alaska                    -0.544444  
Maryland                  -0.477778  
Hawaii                    -0.466667  
Oklahoma                  -0.422222  
Washington                -0.411111